In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re

import warnings
warnings.filterwarnings('ignore')

from flask import Flask
from flask import request
from flask import render_template
import mysql.connector
from mysql.connector import Error
import string

In [2]:
# Connect to MYSQL Database
connection = mysql.connector.connect(host='192.168.12.102',
            database='vonly_data_feed_us_staging',
            user='vonly-agent',
            password='a714fded-311c-4215-8b8b-5df4086e264b')

In [3]:
# for id_1 in v_id:
#     print("\n","Test Case: ",test_case)
#     #print(id_1)
#     test_case=test_case+1
# 444bd446-fff0-479a-b535-859c9a75a796
# 40f43d4d-206a-4430-8232-a7ea89b7fca0
# First SQL Query
query1 = """
SELECT ti.vonly_asset_id,ti.title season_name,ti.portal_item_id season_portal_item_id,ti.num season_no,
                                        t.description,t.distributed_by,t.distributed_by_parent,t.produced_by,t.produced_by_parent,
                                        t.us_theater_release_date,t.director,t.actors,t.writers,t.producers,
                                        t.runtime,t.num season_no2,tie.title episode_name,tie.portal_item_id episode_portal_item_id, tie.num episode_no, ti.platform, 
                                        (select count(1) from tv_ids tii where tii.scope='tv_season' and tii.vonly_asset_id=ti.vonly_asset_id and tii.platform=ti.platform) va_count
                                        FROM vonly_data_feed_us_staging.tv_ids ti
                                        INNER JOIN sandbox.tvshows t ON ti.id=t.vonly_id AND ti.region=t.region 
                                        LEFT OUTER JOIN vonly_data_feed_us_staging.tv_id_mappings tim ON ti.id=tim.season_vonly_id
                                        LEFT OUTER JOIN  vonly_data_feed_us_staging.tv_ids tie ON tim.episode_vonly_id=tie.id  AND tie.scope='tv_episode'
                                         WHERE
                                        ti.scope='tv_season' AND ti.platform NOT IN ('amazon movies & tv') AND  
                                        ti.vonly_asset_id='444bd446-fff0-479a-b535-859c9a75a796'
 
"""

df1 = pd.read_sql_query(query1, connection)

# Second SQL Query
query2 = """
    SELECT ti.vonly_asset_id,ti.title season_name,ti.portal_item_id season_portal_item_id,ti.num season_no,
                                        t.description,t.distributed_by,t.distributed_by_parent,t.produced_by,t.produced_by_parent,
                                        t.us_theater_release_date,t.director,t.actors,t.writers,t.producers,
                                        t.runtime,t.num season_no2,tie.title episode_name,tie.portal_item_id episode_portal_item_id, tie.num episode_no, ti.platform, 
                                        (select count(1) from tv_ids tii where tii.scope='tv_season' and tii.vonly_asset_id=ti.vonly_asset_id and tii.platform=ti.platform) va_count
                                        FROM vonly_data_feed_us_staging.tv_ids ti
                                        INNER JOIN sandbox.tvshows t ON ti.id=t.vonly_id AND ti.region=t.region 
                                        LEFT OUTER JOIN vonly_data_feed_us_staging.tv_id_mappings tim ON ti.id=tim.season_vonly_id
                                        LEFT OUTER JOIN  vonly_data_feed_us_staging.tv_ids tie ON tim.episode_vonly_id=tie.id  AND tie.scope='tv_episode'
                                         WHERE
                                        ti.scope='tv_season' AND ti.platform NOT IN ('amazon movies & tv') AND  
                                        ti.vonly_asset_id='40f43d4d-206a-4430-8232-a7ea89b7fca0'
"""

df2 = pd.read_sql_query(query2, connection)


In [4]:
df1.columns

Index(['vonly_asset_id', 'season_name', 'season_portal_item_id', 'season_no',
       'description', 'distributed_by', 'distributed_by_parent', 'produced_by',
       'produced_by_parent', 'us_theater_release_date', 'director', 'actors',
       'writers', 'producers', 'runtime', 'season_no2', 'episode_name',
       'episode_portal_item_id', 'episode_no', 'platform', 'va_count'],
      dtype='object')

In [5]:
df2.columns

Index(['vonly_asset_id', 'season_name', 'season_portal_item_id', 'season_no',
       'description', 'distributed_by', 'distributed_by_parent', 'produced_by',
       'produced_by_parent', 'us_theater_release_date', 'director', 'actors',
       'writers', 'producers', 'runtime', 'season_no2', 'episode_name',
       'episode_portal_item_id', 'episode_no', 'platform', 'va_count'],
      dtype='object')

In [6]:
df1.head(3)

,vonly_asset_id,season_name,season_portal_item_id,season_no,description,distributed_by,distributed_by_parent,produced_by,produced_by_parent,us_theater_release_date,...,actors,writers,producers,runtime,season_no2,episode_name,episode_portal_item_id,episode_no,platform,va_count
0,444bd446-fff0-479a-b535-859c9a75a796,Band of Brothers and The Pacific,582727129,1,None,HBO,HBO,"Home Box Office, Inc",HBO,2012-12-17,...,None,None,None,None,None,"Band of Brothers, Pt. 1: Currahee",583006787,1,iTunes,1
1,444bd446-fff0-479a-b535-859c9a75a796,Band of Brothers and The Pacific,582727129,1,None,HBO,HBO,"Home Box Office, Inc",HBO,2012-12-17,...,None,None,None,None,None,"Band of Brothers, Pt. 2: Day of Days",583007265,2,iTunes,1
2,444bd446-fff0-479a-b535-859c9a75a796,Band of Brothers and The Pacific,582727129,1,None,HBO,HBO,"Home Box Office, Inc",HBO,2012-12-17,...,None,None,None,None,None,"Band of Brothers, Pt. 3: Carentan",583007737,3,iTunes,1


In [7]:
df2.head(6)

,vonly_asset_id,season_name,season_portal_item_id,season_no,description,distributed_by,distributed_by_parent,produced_by,produced_by_parent,us_theater_release_date,...,actors,writers,producers,runtime,season_no2,episode_name,episode_portal_item_id,episode_no,platform,va_count
0,40f43d4d-206a-4430-8232-a7ea89b7fca0,_DUPE_Big History,B086JKPGKP,1,Change the way you look at the past by answeri...,History,Other,History,Other,2013-11-01,...,None,None,None,None,None,Horse Power Revolution,B086J6S1WW,8,Amazon Prime Video,3
1,40f43d4d-206a-4430-8232-a7ea89b7fca0,_DUPE_Big History,B086JKPGKP,1,Change the way you look at the past by answeri...,History,Other,History,Other,2013-11-01,...,None,None,None,None,None,The Superpower of Salt,B086J6ZQY4,16,Amazon Prime Video,3
2,40f43d4d-206a-4430-8232-a7ea89b7fca0,_DUPE_Big History,B086JKPGKP,1,Change the way you look at the past by answeri...,History,Other,History,Other,2013-11-01,...,None,None,None,None,None,Defeating Gravity,B086JKTJ4F,5,Amazon Prime Video,3
3,40f43d4d-206a-4430-8232-a7ea89b7fca0,_DUPE_Big History,B086JKPGKP,1,Change the way you look at the past by answeri...,History,Other,History,Other,2013-11-01,...,None,None,None,None,None,H2O,B086JL7Q8J,7,Amazon Prime Video,3
4,40f43d4d-206a-4430-8232-a7ea89b7fca0,_DUPE_Big History,B086JKPGKP,1,Change the way you look at the past by answeri...,History,Other,History,Other,2013-11-01,...,None,None,None,None,None,The Sun,B086JL8KC2,15,Amazon Prime Video,3
5,40f43d4d-206a-4430-8232-a7ea89b7fca0,_DUPE_Big History,B086JKPGKP,1,Change the way you look at the past by answeri...,History,Other,History,Other,2013-11-01,...,None,None,None,None,None,The Big History Of Everything,B086JLB2TG,14,Amazon Prime Video,3


# Two query Comparison

In [8]:
mapping_list = []
id_ = 0

for _, row1 in df1.iterrows():
    id_ += 1
    
    count = 0
    mapped = 0
    unmapped = 0
    
    matching_points = {
        'id': id_,
        'title': 0,
        'title_name': row1['season_name'],
        'title_name_1': '',
        'director': 0,
        'director_name': row1['director'],
        'director_name_1':'' ,
        'producers': 0,
        'producer_name': row1['producers'],
        'producer_name_1':'' ,
        'portal_item_id': 0,
        'portal_item_id_': row1['des_portal_id'],
        'portal_item_id_1':'' ,
        'writers': 0,
        'writer_name': row1['writers'],
        'writer_name_1':'' ,
        'runtime': 0,
        'runtime_': row1['runtime'],
        'runtime_1': '',
        'count': count,
        'mapped': mapped,
        'unmapped': unmapped
    }

    for _, row2 in df2.iterrows():
        
        if row1['des_num_epi'] == row2['des_num_epi']:
            if not pd.isnull(row1['season_name']) and not pd.isnull(row2['season_name']) and row1['season_name'] == row2['season_name']:
                matching_points['title_name'] = 1
            elif row1['season_name'] != row2['season_name']:
                matching_points['title_name_1'] = 0
        
            matching_points['title_name_1'] = row2['title']

            if not pd.isnull(row1['director']) and not pd.isnull(row2['director']) and row1['director'] == row2['director']:
                matching_points['director'] = 1
            matching_points['director_name_1'] = row2['director']

            if not pd.isnull(row1['producers']) and not pd.isnull(row2['producers']) and row1['producers'] == row2['producers']:
                matching_points['producers'] = 1
            matching_points['producer_name_1'] = row2['producers']

            if not pd.isnull(row1['des_portal_id']) and not pd.isnull(row2['des_portal_id']) and row1['des_portal_id'] == row2['des_portal_id']:
                matching_points['portal_item_id'] = 1
            elif row1['des_portal_id'] != row2['des_portal_id']:
                matching_points['portal_item_id'] = 0
            matching_points['portal_item_id_1'] = row2['des_portal_id']

            if not pd.isnull(row1['writers']) and not pd.isnull(row2['writers']) and row1['writers'] == row2['writers']:
                matching_points['writers'] = 1
            matching_points['writer_name_1'] = row2['writers']

            if not pd.isnull(row1['runtime']) and not pd.isnull(row2['runtime']) and row1['runtime'] == row2['runtime']:
                matching_points['runtime'] = 1
            matching_points['runtime_1'] = row2['runtime']

            count = matching_points['title'] + matching_points['director'] + matching_points['producers'] + matching_points['portal_item_id'] + matching_points['writers'] + matching_points['runtime']
        

    mapped = 1 if count >= 3 else 0
    unmapped = 1 if count < 3 else 0

    matching_points['count'] = count
    matching_points['mapped'] = mapped
    matching_points['unmapped'] = unmapped
    
    mapping_list.append(matching_points)

mapping_list

KeyError: 'title'

In [19]:
df = pd.DataFrame(mapping_list)

In [20]:
df.to_csv("Mapping.csv")

# 1

# Sort Number

## With Inbuilt

In [26]:
data = [1,10,2,45,0,3]
data.sort()
print(data)

[0, 1, 2, 3, 10, 45]


## Without Inbuilt

In [42]:
data = [1,10,2,45,0,3]
for i in range(0,len(data)):
    for j in range(i+1,len(data)):
        if data[i] > data[j]:
            data[i],data[j] = data[j],data[i]  
print(data)

[0, 1, 2, 3, 10, 45]


# 2

# Armstrong

In [41]:
n = int(input("enter:"))
res = len(str(n))
sum_ = 0
temp = n
while n > 0:
    ans = n % 10
    sum_ = sum_ + ans**res
    n //= 10
if temp == sum_:
    print("Yes")
else:
    print("No")

enter:153
Yes


# 3

# Automorphic

In [1]:
number = int(input("Enter a number:"))
square = pow(number, 2)
mod = pow(10, len(str(number)))

if square % mod == number:
    print("It's an Automorphic Number")
else:
    print("It's not an Automorphic Number")

Enter a number:6
It's an Automorphic Number


# 4

# Int to Roman

# Without Inbuilt

In [4]:
num = int(input("Enter a value: ")) 
symbols = [
    (1000, 'M'),
    (900, 'CM'),
    (500, 'D'),
    (400, 'CD'),
    (100, 'C'),
    (90, 'XC'),
    (50, 'L'),
    (40, 'XL'),
    (10, 'X'),
    (9, 'IX'),
    (5, 'V'),
    (4, 'IV'),
    (1, 'I')
]

roman = ''
for value, symbol in symbols:
    while num >= value:
        roman += symbol
        num -= value

print(roman)

Enter a value: 5
V


# With Inbuilt

In [3]:
import roman

In [5]:
res = roman.toRoman(5)
print(res)

V


# 5

# Roman to INT

##  With Inbuilt

In [6]:
import roman

In [7]:
res = roman.fromRoman('V')
print(res)

5


# Without inbuilt

In [36]:
num = int(input("Enter a value: "))  # Convert input to an integer
m = ["", "M", "MM", "MMM"]
c = ["", "C", "CC", "CCC", "CD", "D",
     "DC", "DCC", "DCCC", "CM "]
x = ["", "X", "XX", "XXX", "XL", "L",
     "LX", "LXX", "LXXX", "XC"]
i = ["", "I", "II", "III", "IV", "V",
     "VI", "VII", "VIII", "IX"]

thousands = m[num // 1000]
hundreds = c[(num % 1000) // 100]
tens = x[(num % 100) // 10]
ones = i[num % 10]

ans = (thousands + hundreds +
       tens + ones)
print(ans)

Enter a value: 10
X


# 6

# Binary to decimal

In [8]:
# Both
n = 4
binary = format(n, 'b')
print(binary) # Output: '100'
binary = '100'
decimal = int(binary, 2)
print(decimal) # Output: 4

100
4


In [12]:
n = int(input("Enter a value: "))
decimal, i = 0, 0
while( n != 0):
    dec = n % 10
    decimal = decimal + dec * pow(2, i)
    n = n//10
    i += 1
print(decimal)

Enter a value: 1010
10


# Decimal to Binary

In [13]:
decNum = 10
print(bin(decNum)[2:])

1010


In [37]:
n = int(input("Enter a value: "))

binary = ""  

while n > 0:
    remainder = n % 2  
    binary = str(remainder) + binary 
    n = n // 2  

print("Binary representation:", binary)


Enter a value: 10
Binary representation: 1010
